In [1]:
from azure.quantum import Workspace
workspace = Workspace (
    subscription_id = "b1d7f7f8-743f-458e-b3a0-3e09734d716d",
    resource_group = "aq-hackathons",
    name = "aq-hackathon-01",
    location = "eastus"
)

In [2]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from azure.quantum.qiskit import AzureQuantumProvider
from qiskit import execute
from qiskit.circuit import Parameter
import numpy as np
import qiskit
from qiskit.quantum_info import Pauli
from qiskit import opflow
from qiskit.opflow import PauliSumOp

### 2-site system
### (1/2) p12 x1 x2 + (1/2) p21 x2 x1

### (1/2) p12 (1 + Z1) / 2 (1 + Z2) / 2 + (1/2) p12 (1 + Z1) / 2 (1 + Z2) / 2
### p12 / 4 + Z1 (1 / 4 p12) + Z2 (1 / 4 p12)


# \sum_ij Wij xi xj
# xi/xj = 0, 1
# xi -> (1 + Zi) / 2, xj -> (1 + Zj) / 2
# pi xi^2 -> pi [(1 + Zi) / 2]^2 = pi (1 + Zi) / 2 = pi/2 + pi Zi / 2
# (1/2) \sum_ij pij xi xj ->pij x (1 + Zi) / 2 x (1 + Zj) / 2 = pij/4 + pij Zi/4 + pij Zj/4 + pijZi Zj / 4

# on diagonal of W, stay resulting coefficients in front of Z_i
# on off-diagonal of W, stay resu. coefficients in front of Z_i Z_j, but also divided by 2

def to_matrix(onsite, pair, n_qubits):
    W = np.zeros((n_qubits, n_qubits), dtype=np.float64)
    for onsite_term in onsite:
        i, pi = onsite_term
        W[i, i] += pi / 2.
        
    for pair_term in pair:
        i, j, pij = pair_term
        W[i, j] += pij / 8.
        W[j, i] += pij / 8
            
        W[i, i] += pij / 8
        W[j, j] += pij / 8
        
    return W


def transform_interaction_to_qiskit_format(n_qubits, hamiltonian):
    r"""Generate Hamiltonian for the problem
    """
    
    onsite = hamiltonian.onsite
    pair = hamiltonian.pair
    
    def get_shift(onsite, pair):
        shift = 0.
        for onsite_term in onsite:
            _, pi = onsite_term
            
            shift += pi / 2.
            
        for pair_term in pair:
            _, _, pij = pair_term
            
            shift += pij / 4.
        return shift

    shift = get_shift(onsite, pair)
    
    W = to_matrix(onsite, pair, n_qubits)
    
    pauli_list = []

    for i in range(n_qubits):
        for j in range(n_qubits):
            if np.isclose(W[i, j], 0.0):
                continue
            x_p = np.zeros(n_qubits, dtype=bool)
            z_p = np.zeros(n_qubits, dtype=bool)
            z_p[i] = True
            z_p[j] = True
            pauli_list.append([W[i, j], Pauli((z_p, x_p))])

    pauli_list = [(pauli[1].to_label(), pauli[0]) for pauli in pauli_list]
    return PauliSumOp.from_list(pauli_list), shift

def evaluate_cost(solution, hamiltonian):
    energy = 0
    for single_term in hamiltonian.onsite:
        energy += single_term[1] * (solution[single_term[0]] == 1)
        
    for pair_term in hamiltonian.pair:
        energy += pair_term[2] * (solution[pair_term[0]] == 1) * (solution[pair_term[1]] == 1)
        
    return energy


def index_to_spin(index, n_qubits):
    return (((np.array([index]).reshape(-1, 1) & (1 << np.arange(n_qubits)))) > 0).astype(np.int64)

def bruteforce_solution(n_qubits, hamiltonian):
    energies = []
    bit_representations = []
    for idx in range(2 ** n_qubits):
        solution = index_to_spin(idx, n_qubits)[0]
        
        
        bit_representations.append(solution.copy())
        energies.append(evaluate_cost(solution, hamiltonian))
        
    energies = np.array(energies)
    bit_representations = np.array(bit_representations)
    
    return np.sort(energies), bit_representations[np.argsort(energies)]



def maxcut_obj(x, G):
    """
    Given a bitstring as a solution, this function returns
    the number of edges shared between the two partitions
    of the graph.
    
    Args:
        x: str
           solution bitstring
           
        G: networkx graph
        
    Returns:
        obj: float
             Objective
    """
    obj = 0
    for i, j in G.edges():
        if x[i] != x[j]:
            obj -= 1
            
    return obj


def compute_expectation(counts, ham):
    """
    Computes expectation value based on measurement results
    
    Args:
        counts: dict
                key as bitstring, val as count
        ham: hamiltonian instance
    Returns:
        avg: float
             expectation value
    """
    
    def cast_to_int_array(bitstring):
        return np.asarray([int(y) for y in (list(bitstring))])
    
    avg = 0
    sum_count = 0
    for bitstring, count in counts.items():
        obj = evaluate_cost(cast_to_int_array(bitstring), ham) ## BEWARE: order of bitstring
        avg += obj * count
        sum_count += count
    #print('CURRENT LOSS:', avg / sum_count)
    return avg/sum_count


# We will also bring the different circuit components that
# build the qaoa circuit under a single function
def create_qaoa_circ(ham, theta):
    #print('CURRENT PARAMETERS DURING THE OPTIMIZATION ARE:', theta)
    """
    Creates a parametrized qaoa circuit
    
    Args:  
        ham: networkx graph
        theta: list
               unitary parameters
                     
    Returns:
        qc: qiskit circuit
    """
    
    nqubits = ham.n_qubits
    p = len(theta) // 2  # number of alternating unitaries
    qc = QuantumCircuit(nqubits)

    beta = theta[:p]
    gamma = theta[p:]  # TODO add rz parameters
    
    # initial_state
    for i in range(nqubits):
        qc.h(i)
    
    for irep in range(p):
        # problem unitary
        for pair in list(ham.pair):
            qc.rzz(2 * gamma[irep] * pair[2], pair[0], pair[1])
            
        for onsite in list(ham.onsite):
            qc.rz(2 * gamma[irep] * onsite[1], onsite[0])

        # mixer unitary
        for i in range(nqubits):
            qc.rx(2 * beta[irep], i)
            
    qc.measure_all()
        
    return qc

# Finally we write a function that executes the circuit on the chosen backend
def get_expectation(ham, backend ,p, shots=512):
    
    """
    Runs parametrized circuit
    
    Args:
        hham: hamiltonian
        p: int,
           Number of repetitions of unitaries
    """
    
    backend.shots = shots
    
    def execute_circ(theta):
        qc = create_qaoa_circ(ham, theta)
        print('Starting iteration')
        counts = backend.run(qc).result().get_counts()
        print('Completed iteration')
        return compute_expectation(counts, ham)
    
    return execute_circ

In [3]:
from collections import OrderedDict
from qiskit import algorithms, transpile
from qiskit.algorithms import QAOA
from qiskit.opflow import StateFn
from qiskit.algorithms.optimizers import ADAM, COBYLA
from qiskit.circuit.library import TwoLocal
from qiskit.algorithms import VQE
from qiskit.circuit.library import TwoLocal


def most_frequent_strings(state_vector, num_most_frequent):
    """Compute the most likely binary string from state vector.
    Args:
        state_vector (numpy.ndarray or dict): state vector or counts.
    Returns:
        numpy.ndarray: binary string as numpy.ndarray of ints.
    """
    most_frequent_strings = [x[0] for x in sorted(state_vector.items(), \
                                                  key=lambda kv: kv[1])[-num_most_frequent:]]
    return [np.asarray([int(y) for y in (list(binary_string))]) for binary_string in most_frequent_strings]




class hamiltonian(object):
    def __init__(self, onsite, pair, n_qubits):
        self.onsite = onsite
        self.pair = pair
        self.n_qubits = n_qubits
        return


def get_random_Hamiltonian(n_qubits):
    onsite = []
    pair = []
    
    for i in range(n_qubits):
        onsite.append((i, np.random.uniform(-2, 2)))
        
    for i in range(n_qubits):
        for j in range(i + 1, n_qubits):
            pair.append((i, j, np.random.uniform(-2, 2)))
    return hamiltonian(onsite, pair, n_qubits)

# ham = get_random_Hamiltonian(n_qubits)
n_qubits = 4
onsite = [(0,-2),(1,3),(2,0),(3,-3)]
pair = [(0,1,1),(0,2,-1.5),(0,3,-1),(1,2,-2)]

ham = hamiltonian(onsite, pair, n_qubits)

energies, bits = bruteforce_solution(n_qubits, ham)

print('ALL BRUTE FORCE SOLUTIONS')

for en, xi in zip(energies, bits):
    print('BF string:', xi, 'cost:', en)



provider = AzureQuantumProvider(
    resource_id = "/subscriptions/b1d7f7f8-743f-458e-b3a0-3e09734d716d/resourceGroups/aq-hackathons/providers/Microsoft.Quantum/Workspaces/aq-hackathon-01",
    location = "eastus"
)

backend = provider.get_backend( 'ionq.simulator' )

backend.shots = 512


p = n_qubits

from scipy.optimize import minimize
expectation = get_expectation(ham, backend, p=p)

#res = minimize(expectation, np.ones(2 * p), method='COBYLA', options = {'maxiter':40})
#print('FINAL PARAMETERS:', res.x)
# theta_opt = res.x

theta_opt = np.array([0.76612481, 0.92081674, 1.1441194,  1.30785258, 1.36184247, 2.23596843, 0.95932347, 0.91564337])

qc_res = create_qaoa_circ(ham, theta_opt)


qc_trans = transpile(qc_res, backend, optimization_level=3)
qc_trans.depth
print(qc_res)
print(qc_trans)

#backend =  provider.get_backend( 'ionq.simulator' )
backend =  provider.get_backend( 'ionq.qpu' )
backend.shots = 20

counts = backend.run(qc_res).result().get_counts()
print(counts)
x = most_frequent_strings(counts, 4)
#x = [1 - xi[::-1] for xi in x]


print('\n\n\nTESTING THE QUANTUM OUTPUT')
for xi in x:
    print('QC string:', xi, 'cost:', evaluate_cost(xi, ham))





#qubit_op, offset = transform_interaction_to_qiskit_format(n_qubits, ham)


#optimizer = COBYLA()
#
#vqe = QAOA(optimizer, quantum_instance=Aer.get_backend('qasm_simulator'))# 
#ansatz = TwoLocal(qubit_op.num_qubits, 'ry', 'cz', reps=5, entanglement='full')
#vqe = VQE(ansatz, optimizer, quantum_instance=Aer.get_backend('qasm_simulator'))

#result = vqe.compute_minimum_eigenvalue(qubit_op)



ALL BRUTE FORCE SOLUTIONS
BF string: [1 0 1 1] cost: -7.5
BF string: [1 0 0 1] cost: -6.0
BF string: [1 1 1 1] cost: -5.5
BF string: [1 0 1 0] cost: -3.5
BF string: [0 0 0 1] cost: -3.0
BF string: [0 0 1 1] cost: -3.0
BF string: [1 0 0 0] cost: -2.0
BF string: [1 1 0 1] cost: -2.0
BF string: [0 1 1 1] cost: -2.0
BF string: [1 1 1 0] cost: -1.5
BF string: [0 0 0 0] cost: 0.0
BF string: [0 0 1 0] cost: 0.0
BF string: [0 1 0 1] cost: 0.0
BF string: [0 1 1 0] cost: 1.0
BF string: [1 1 0 0] cost: 2.0
BF string: [0 1 0 0] cost: 3.0
        ┌───┐                                         ┌─────────────┐»
   q_0: ┤ H ├─■────────────■─────────────■────────────┤ Rz(-5.4474) ├»
        ├───┤ │ZZ(2.7237)  │             │            └─────────────┘»
   q_1: ┤ H ├─■────────────┼─────────────┼──────────────■────────────»
        ├───┤              │ZZ(-4.0855)  │              │ZZ(-5.4474) »
   q_2: ┤ H ├──────────────■─────────────┼──────────────■────────────»
        ├───┤                            │

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
print(counts)
wave_vector = pd.DataFrame.from_dict(counts, orient='index', columns = ['count'])
print(wave_vector)
print(wave_vector.index)

In [ ]:
fig, ax = plt.subplots()

# Save the chart so we can loop through the bars below.
bars = ax.bar(
    x=np.arange(wave_vector.size),
    height=wave_vector['count'],
    tick_label=wave_vector.index
)

ax.xticks(rotation=60)

# Axis formatting.
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_color('#DDDDDD')
ax.tick_params(bottom=False, left=False)
ax.set_axisbelow(True)
ax.yaxis.grid(True, color='#EEEEEE')
ax.xaxis.grid(False)

# Add text annotations to the top of the bars.
bar_color = bars[0].get_facecolor()
for bar in bars:
  ax.text(
      bar.get_x() + bar.get_width() / 2,
      bar.get_height() + 0.3,
      round(bar.get_height(), 1),
      horizontalalignment='center',
      color=bar_color,
      weight='bold'
  )



# Add labels and a title. Note the use of `labelpad` and `pad` to add some
# extra space between the text and the tick labels.
ax.set_xlabel('Basis states', labelpad=15, color='#333333')
ax.set_ylabel('Absolute frequency', labelpad=15, color='#333333')
ax.set_title('State vector counts [500 shots]', pad=15, color='#333333',
             weight='bold')

fig.tight_layout()


In [ ]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline


class hamiltonian(object):
    def __init__(self, onsite, pair, n_qubits):
        self.onsite = onsite
        self.pair = pair
        self.n_qubits = n_qubits
        return


def get_random_Hamiltonian(n_qubits):
    onsite = []
    pair = []
    
    for i in range(n_qubits):
        onsite.append((i, np.random.uniform(-2, 2)))
        
    for i in range(n_qubits):
        for j in range(i + 1, n_qubits):
            pair.append((i, j, np.random.uniform(-2, 2)))
    return hamiltonian(onsite, pair, n_qubits)

# ham = get_random_Hamiltonian(n_qubits)
n_qubits = 4
onsite = [(0,-2),(1,3),(2,0),(3,-3)]
pair = [(0,1,1),(0,2,-1.5),(0,3,-1),(1,2,-2)]

ham = hamiltonian(onsite, pair, n_qubits)

counts = {'0000': 14, '0001': 19, '0010': 22, '0011': 4, '0100': 18, '0101': 21, '0110': 60, '0111': 8, '1000': 28, '1001': 111, '1010': 31, '1011': 24, '1100': 12, '1101': 29, '1110': 42, '1111': 57}
cost_list = []
for xi in range(2**n_qubits):
    solution = index_to_spin(xi, n_qubits)[0]
    cost_list.append(evaluate_cost(solution, ham))


wave_vector = pd.DataFrame.from_dict(counts, orient='index', columns = ['count'])
wave_vector['costs'] = cost_list


# set font
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Helvetica'

# set the style of the axes and the text color
plt.rcParams['axes.edgecolor']='#333F4B'
plt.rcParams['axes.linewidth']=0.8
plt.rcParams['xtick.color']='#333F4B'
plt.rcParams['ytick.color']='#333F4B'
plt.rcParams['text.color']='#333F4B'

df = wave_vector
print(df)
df = df.sort_values(by=['costs'])
print(df)

# we first need a numeric placeholder for the y axis
my_range=list(range(1,len(df.index)+1))

fig, ax = plt.subplots(figsize=(5,3.5))

# create for each expense type an horizontal line that starts at x = 0 with the length 
# represented by the specific expense percentage value.
plt.hlines(y=my_range, xmin=0, xmax=df['count'], color='#007ACC', alpha=0.2, linewidth=5)

# create for each expense type a dot at the level of the expense percentage value
plt.plot(df['count'], my_range, "o", markersize=5, color='#007ACC', alpha=0.6)

# set labels
ax.set_xlabel('Counts', fontsize=15, fontweight='black', color = '#333F4B')
ax.set_ylabel('')

# set axis
ax.tick_params(axis='both', which='major', labelsize=12)
plt.yticks(my_range, df.index)

# add an horizonal label for the y axis 
fig.text(-0.23, 0.96, 'Basis state', fontsize=15, fontweight='black', color = '#333F4B')

# change the style of the axis spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.spines['left'].set_bounds((1, len(my_range)))
ax.set_xlim(0,max(df['count'])+2)

ax.spines['left'].set_position(('outward', 8))
ax.spines['bottom'].set_position(('outward', 5))


plt.show()
plt.savefig('hist_qpu_500shots.png', dpi=300, bbox_inches='tight')
